In [13]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import datetime
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold=np.inf)

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [14]:
data = pd.read_excel('F&F.xlsx')
data.shape

(6060, 173)

In [15]:
data.head()

,ORG_POSITION,PARTICIPATION_TYPE,PARTICIPATION_FREQUENCY,SENT_DATE,Дата в формате текст,45115,Года,Статус участника,Количество счетов в других банках,Эквайринг_терминал,Контакт_выдан_ответа_нет,Дата_недозвон_1,Дата_недозвон_2,Дата_недозвон_3,Д_посл_дозвона,Д_посл_участия,Формат_участия,Д_посл_участия_в_глубинке,Формат_участия_перечислить,Количество участий в исследованиях,AR_REVENUE,ACCOUNT_OPEN_DT,Дата открытия в EGR,Дата открытия в витрине с пакетами Сбера,ACTIVE_CARDS_CNT,CU_ADDR_COMP,active_flag,MAIN_OKVED_NM,OKOPF_NM,REGISTRATION_DT,CRM_SEGMENT_TYPE_NM,CRM_CA_TB_NM,Откр. счета без дог-конструктора,Откр. счета с дог-конструктором,Платформе Деловая Среда,Сопровождение контрактов,Прием наличных на счет через УС,E-invoicing,Защита сотрудников,Сберкурьер,Эл. подпись для торгов (федер),Доставка денежной наличности,Моя бухгалтерия Аутсорсинг,Внесение изменений в док,СМС-информирование по рс,Юр поддержка,Страхование корп карт для ИП,Юрист для бизнеса,Сбереги здоровье +,Сбереги бизнес,Мобильная связь Бизнес,Сервис проверки контрагентов,СМС-информ по БКе,Корпоративное кредитование-Прото,Решение для клиентов ВЭД (рубли),Овердрафт с общим лимитом-Прото,Сбер Решения-Бухгалтерия для ИП,GM. Открытие брокерского счета,Эвотор-Покупка смарт-терминала,Сбербанк Бизнес Онлайн (СББОЛ),(МБ) Кредитование по рег №279,Эвотор-Аренда смарт-терминала,2216/Бизнес лимит без сделки,Прием по объявлению на взнос,Факторинг без права регресса,СМАРТ кредитная бизнес-карта,Пакет услуг Активные расчеты,Предзаявка в Сбербанк Лизинг,Пакет услуг Хорошая выручка,Корпоративное кредитование,СберМаркет-СберМаркет B2B,Корус-Электронная подпись,СберЛогистика-Сбер Курьер,Безлимитная самоинкасация,СберЗдоровье-Телемедицина,Пакет услуг Удачный сезон,Моментальная бизнес-карта,Корус-Сфера курьер (ЭДО),2216/Бизнес-недвижимость,Остатки на РТС (без НСО),Премиальная бизнес-карта,Консультационные услуги,Прием платежей физ. лиц,1948/Оборотный (Москва),Сбер Кредо-(ИП Заемщик),Сбер Кредо-(ЮЛ Заемщик),Страхование бизнес-карт,1948/Экспресс-овердрафт,Юрист для бизнеса РКО +,Моя бухгалтерия Онлайн,Документарные операции,ПУ Большие возможности,Консалтинг по ВЭД и ВК,Кредитная бизнес-карта,Конверсионные операции,Цифровая бизнес карта,2216/Бизнес-овердрафт,Залоговое страхование,ДМС. Сбереги здоровье,Защита ваших успехов,Реструктуризация 1/3,Реструктуризация ММБ,Спец счет для торгов,2216/Бизнес-контракт,2216/Бизнес-гарантия,Электронная подпись,2216/Инвестиционный,2216/Бизнес-доверие,Переводы физ. лицам,Депозитарные услуги,Работа ру-Платформа,Безлимитные платежи,Бизнес-класс Google,Банковская гарантия,Неснижаемый остаток,Экспресс-овердрафт,2216/Бизнес-инвест,Интернет-эквайринг,2216/Бизнес-оборот,ПУ Набирая обороты,Зарплатные проекты,Депозиты (срочные),2216/Бизнес-актив,ПУ ВЭД без границ,Стабильный бизнес,Расчетные векселя,Оплата по QR коду,Валютный контроль,Доходные векселя,Корус-Рейтинг ЮЛ,ПУ Полным ходом,Cash Management,Бюджетная карта,СМАРТ-овердрафт,Эвотор-На_Полке,ПУ Минимальный+,Rabota.ru РКО +,ПУ Легкий старт,2216/Оборотный,1С бухгалтерия,Конструктор ПУ,1948/Овердрафт,Страхование НС,1948/Оборотный,GM. E-TRADING,ПУ Финансовый,Смарт-POS 2в1,2216/Гарантия,Навигатор МСП,СМАРТ-оборот,СМАРТ-тендер,Доктор рядом,1948/Доверие,Кредитование,СМАРТ-кредит,Бизнес-карта,GM. FX SWAP,Аккредитивы,Страхование,ПУ Оптима+,Инкассация,GM. Notes,ПУ Актив+,ПУ Базис+,Эквайринг,GM. REPO,GM. CAP,Векселя,GM. FWD,"Клиент использовал продукт ""Лизинг"" хотя бы раз за последние три месяца, начиная с 31 марта 2022 года",Льготное кредитование,Самоинкассация,Доступны реестровые выплаты СМЗ (по состоянию на 22.08.2022)
0,NaN,Личная встреча,Не чаще 1-2 раз в год,9 августа 2021,44418,697,1.909589,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20936000,2004-09-09,2016-01-31 00:00:00,2004-09-09 00:00:00,10.0,"392035, Тамбовская область, г.Тамбов, ул. Монтажников, дом 10, корп. Лит А",1.0,"ПРОИЗВОДСТВО СТРОИТЕЛЬНЫХ МЕТАЛЛИЧЕСКИХ КОНСТРУКЦИЙ, ИЗДЕЛИЙ И ИХ ЧАСТЕЙ",ООО,2000-12-08,Микро,Центрально-Черноземный банк,0.0,0.0,0.0,

In [16]:
data = data.loc[data['ORG_POSITION'].str.contains(r'\w*(?:test|тест)\w*',
                                           case=False,
                                           regex=True) == False]

In [17]:
data['ORG_POSITION'].nunique()

1019

In [18]:
data.dtypes.unique()

array([dtype('O'), dtype('int64'), dtype('float64'), dtype('<M8[ns]')],
      dtype=object)

In [19]:
obj_features = list(data.select_dtypes(include=['object']).columns)
num_features = list(data.select_dtypes(include=['float64', 'int64']).columns)

data['CRM_SEGMENT_TYPE_NM'] = data['CRM_SEGMENT_TYPE_NM'].fillna('сегмент не задан')
data[num_features] = data[num_features].fillna(0)
data[obj_features] = data[obj_features].fillna('')

data['AR_REVENUE'] = pd.to_numeric(data['AR_REVENUE'], errors='coerce').fillna(0)

In [20]:
data_analyze = data[['ORG_POSITION', 'PARTICIPATION_TYPE', 'PARTICIPATION_FREQUENCY', 'SENT_DATE', 'Статус участника', 
                     'Количество счетов в других банках', 'Эквайринг_терминал', 'Контакт_выдан_ответа_нет',
                     'Количество участий в исследованиях', 'AR_REVENUE', 'ACCOUNT_OPEN_DT',
                     'ACTIVE_CARDS_CNT', 'active_flag', 'MAIN_OKVED_NM', 'CRM_SEGMENT_TYPE_NM']].copy()

In [21]:
def convert_day_num(str):
    if re.search("^\d+$", str) or re.search("^\d+\.\d+$", str):
        return datetime.date(1899, 12, 30) + datetime.timedelta(days=int(float(str)))
    return str

def replace_text_month(s):
    return s.replace(
                    to_replace=['\sянваря\s', '\sфевраля\s', '\sмарта\s', '\sапреля\s', '\sмая\s', '\sиюня\s', 
                        '\sиюля\s', '\sавгуста\s', '\sсентября\s', '\sоктября\s', '\sноября\s', '\sдекабря\s'], 
                    value=['.01.', '.02.', '.03.', '.04.', '.05.', '.06.', 
                            '.07.', '.08.', '.09.', '.10.', '.11.', '.12.'], 
                    regex=True)
    
def process_date_text_column(col_name, text_exists=False, df=data_analyze):
    df[col_name] = replace_text_month(data[col_name].apply(convert_day_num))
    
    if text_exists:
        df[f'{col_name}_c'] = df[col_name].str.extract(r"([А-Яа-я!,.:;\s]+)", expand=True)
        df[f'{col_name}_c'].fillna('')
        df[col_name] = df[col_name].str.extract(r"(\d{2}.\d{2}.\d{4})", expand=True)
    
    df[col_name] = pd.to_datetime(df[col_name], format='mixed', errors = 'coerce').dt.normalize()
    # df[col_name] = pd.to_datetime(df[col_name], format='%Y-%m-%d', errors = 'coerce')
   
    
process_date_text_column('SENT_DATE')

In [22]:
data_analyze['SENT_DATE'].min()

Timestamp('2020-05-29 00:00:00')

In [23]:
data_analyze['SENT_DATE'].max()

Timestamp('2023-05-30 00:00:00')

In [24]:
data_analyze['SENT_DATE'].nunique()

812

In [25]:
data_analyze['SENT_DATE'].dt.to_period('M').astype('str').value_counts().sort_index()

SENT_DATE
2020-05      2
2020-06    131
2020-07    116
2020-08    535
2020-09    929
2020-10    225
2020-11    628
2020-12    250
2021-01     74
2021-02    245
2021-03    193
2021-04    196
2021-05    149
2021-06    178
2021-07    140
2021-08    157
2021-09    212
2021-10    121
2021-11    175
2021-12    140
2022-01     85
2022-02    115
2022-03     34
2022-04     60
2022-05     50
2022-06    102
2022-07    124
2022-08     68
2022-09     54
2022-10     96
2022-11     67
2022-12     35
2023-01     48
2023-02     70
2023-03     75
2023-04     93
2023-05     77
NaT          1
Name: count, dtype: int64

In [26]:
indexes = data['CU_ADDR_COMP'].str.extract(r"(\w*\d{6}\w*)", expand=True)
indexes['code'] = indexes[0].transform(lambda x: x.str[:3])
indexes.loc[~indexes.isnull()[0], :] = indexes.loc[~indexes.isnull()[0], :].astype('int')
indexes.drop(columns=[0], inplace=True)

nan_rows = indexes.loc[indexes['code'].isnull()].index.to_list()

regions = data.loc[nan_rows, 'CU_ADDR_COMP'].str.lower().str.extract(
    r"((?:ханты-мансийский автономный округ|республика саха \(якутия\)|республика северная осетия — алания|москва|санкт-петербург|с.-петербург|севастополь|[а-яА-Я-]*\sобл[а-яА-Я]*|[а-яА-Я- ]*край(?:[а-яА-Я-]|\s–|–)*|[а-яА-Я- ]*респ(?:[а-яА-Я- ]|\s–|–)*|[а-яА-Я-]*округ(?:[а-яА-Я -]|\s–|–)*))", 
    expand=True)
regions[0] = regions[0].str.strip()

regions_with_codes = pd.read_html('https://shuchkin.ru/2020/01/kody-regionov-rossii-i-pochtovye-indeksy-region-po-indeksu/', match='Почта')[0]
regions_with_codes = pd.DataFrame(regions_with_codes[[0, 3]].values[1:], columns=regions_with_codes[[0, 3]].values[0])
regions_with_codes['Регион'] = regions_with_codes['Регион'].str.lower()
regions_with_codes['Почта'] = regions_with_codes['Почта'].apply(lambda x: [int(x[i:(i+3)]) for i in range(0, len(x), 3)])
regions_with_codes = regions_with_codes.explode('Почта')

dict_regions_codes = pd.Series(regions_with_codes['Регион'].values, index=regions_with_codes['Почта'].values).to_dict()

regions.rename(columns={0:'code'}, inplace=True)

indexes['code'] = indexes['code'].map(dict_regions_codes, na_action='ignore')
indexes.loc[regions.index, 'code'] = regions['code'].values.copy()

indexes.loc[(indexes['code'] == "стопиньский край") | (indexes['code'] == "республика сербия"), 'code'] = np.nan

In [27]:
data_analyze['Region'] = indexes['code'].str.title()
data_analyze.head()

,ORG_POSITION,PARTICIPATION_TYPE,PARTICIPATION_FREQUENCY,SENT_DATE,Статус участника,Количество счетов в других банках,Эквайринг_терминал,Контакт_выдан_ответа_нет,Количество участий в исследованиях,AR_REVENUE,ACCOUNT_OPEN_DT,ACTIVE_CARDS_CNT,active_flag,MAIN_OKVED_NM,CRM_SEGMENT_TYPE_NM,Region
1,Директор,Телефонные интервью,Не чаще 1 раза в месяц,2021-10-08,1,0.0,,0.0,0,0.0,NaT,0.0,0.0,,сегмент не задан,NaN
2,Директор,Телефонные интервью,Без ограничений,2021-02-09,1,0.0,,0.0,1,0.0,2015-06-10,2.0,1.0,ДЕЯТЕЛЬНОСТЬ РЕСТОРАНОВ И УСЛУГИ ПО ДОСТАВКЕ ПРОДУКТОВ ПИТАНИЯ,Малые,Ханты-Мансийский Автономный Округ – Югра
3,Индивидуальный предприниматель,Телефонные интервью,Не чаще 1-2 раз в год,2021-02-09,1,0.0,,0.0,1,0.0,2021-04-08,0.0,1.0,состоялась,Малые,Алтайский Край
4,Директор,Телефонные интервью,Не чаще 1 раза в месяц,2021-02-09,1,0.0,,0.0,1,1740000.0,2019-01-17,0.0,1.0,ДЕЯТЕЛЬНОСТЬ ПО КОМПЛЕКСНОМУ ОБСЛУЖИВАНИЮ ПОМЕЩЕНИЙ,Микро,Пермский Край
5,Генеральный директор,"Online-интервью (zoom, skype и др.);Личная встреча",Без ограничений,2021-02-09,1,0.0,,0.0,1,5060000.0,NaT,0.0,1.0,ДЕЯТЕЛЬНОСТЬ РЕКЛАМНАЯ,Малые,NaN


In [28]:
region_exists_num = data.shape[0] - data['CU_ADDR_COMP'].value_counts()[0]
data_analyze['Region_freq'] = data_analyze.groupby('Region')['Region'].transform('count')
data_analyze['Region_percentage (%)'] = data_analyze['Region_freq'].transform(lambda x: round(x*100/region_exists_num, 2))

In [29]:
conditions = [data_analyze['ORG_POSITION'].str.fullmatch(
      r'\s*(?:(?!\s*финанс|\s*эконом).*\s*(?:[а-я,.\'\"-]|\s)*(?:директор\s*|дир\.*\s*)|(?:[а-я,.\'\"-]|\s)*ип(?:[а-я,.()\'\"-]|\s)*|(?:[а-я,.()-]|\s)*руководитель(?:[а-я,.()-]|\s)*|(?:[а-я,.()-]|\s)*предприниматель(?:[а-я,.()-]|\s)*|CEO|(?:[а-я,.()-]|\s)*председатель(?:[а-я,.()-]|\s)*|(?:[а-я,.()-]|\s)*учредитель(?:[а-я,.()-]|\s)*|(?:[а-я,.()-]|\s)*индивидуальный предприниматель(?:[а-я,.()-]|\s)*|(?:[а-я,.()-]|\s)*собственник(?:[а-я,.()-]|\s)*|(?:[а-я,.()-]|\s)*владелец(?:[а-я,.()-]|\s)*|руководитель\s*[а-я,()]*|boss|босс|(?:заместитель|зам\.*)\s*(?:ген[а-я]*\.*||ком[а-я]*)*(?!финанс|эконом).\s*дир[а-я]*\.*\s*|ооо)\s*',
        case=False),
              
              data_analyze['ORG_POSITION'].str.fullmatch(
      r'\s*(?:(?:[а-я,.-]|\s)*(?:финанс|экономич|экономик|казначей)(?:[а-я,.-]|\s)*).*\s*',
        case=False),
              
              data_analyze['ORG_POSITION'].str.fullmatch(
      r'\s*(?:[а-я.,-]*\s*экономист|[а-я.,-]*\s*бух[а-я.,-]*|экономист-бухгалтер|бухгалтер-экономист)+\s*',
        case=False)
]

outputs = ['Генеральный директор', 'Финансовый директор', 'Бухгалтер']
              
data_analyze['org_type'] = np.select(conditions, outputs, default='Другой тип сотрудника')

In [30]:
data_analyze.loc[:, ['ORG_POSITION', 'org_type']].head(10)

,ORG_POSITION,org_type
1,Директор,Генеральный директор
2,Директор,Генеральный директор
3,Индивидуальный предприниматель,Генеральный директор
4,Директор,Генеральный директор
5,Генеральный директор,Генеральный директор
6,Исполнительный директор,Генеральный директор
7,"Собственник, руководитель",Генеральный директор
8,ИП,Генеральный директор
9,Генеральный директор,Генеральный директор
10,Владелец,Генеральный директор


In [31]:
data_analyze['org_type'].value_counts()

org_type
Генеральный директор     4662
Другой тип сотрудника     678
Бухгалтер                 583
Финансовый директор       127
Name: count, dtype: int64

In [32]:
data_analyze['MAIN_OKVED_NM'] = data_analyze['MAIN_OKVED_NM'].str.capitalize()

data_analyze = data_analyze.assign(участвовали1 = data_analyze['Количество участий в исследованиях'] == 1).\
    assign(участвовали2 = data_analyze['Количество участий в исследованиях'] == 2).\
    assign(участвовали3 = data_analyze['Количество участий в исследованиях'] == 3).\
    assign(участвовали4 = data_analyze['Количество участий в исследованиях'] == 4)
    
for num in range(1, 5):
    data_analyze[f'Участ. {num} по отрасли'] = data_analyze.groupby('MAIN_OKVED_NM')[f'участвовали{num}'].transform('sum')
    
data_analyze['Участ. по отрасли'] = data_analyze.loc[:, ['Участ. 1 по отрасли', 'Участ. 2 по отрасли', 'Участ. 3 по отрасли', 'Участ. 4 по отрасли']].sum(axis=1)
data_analyze['Кол. заявок по отрасли'] = data_analyze.groupby('MAIN_OKVED_NM')['MAIN_OKVED_NM'].transform('count')

data_industry = data_analyze.loc[:,
    ['MAIN_OKVED_NM', 'Участ. 1 по отрасли', 'Участ. 2 по отрасли',	'Участ. 3 по отрасли',	'Участ. 4 по отрасли',	
     'Участ. по отрасли', 'Кол. заявок по отрасли']].drop_duplicates()
data_industry.shape

(739, 7)

In [33]:
data_industry.to_excel('F&F_по_отраслям.xlsx')

In [36]:
regions_data = data_analyze.loc[:, ['Region','Region_freq', 'Region_percentage (%)']].drop_duplicates()

In [37]:
regions_data.to_excel('F&F_по_регионам.xlsx')

In [38]:
data_analyze.drop(columns=['участвовали1', 'участвовали2', 'участвовали3', 'участвовали4'], inplace=True)
regions_data.to_excel('F&F_изменный.xlsx')